In [2]:
import sys
sys.path.append('../')
%reload_ext autoreload
%autoreload 2

import os
from os.path import join as pjoin

import utils.paramUtil as paramUtil
from options.train_options import TrainCompOptions
from utils.plot_script import *

from models import MotionTransformer, MaskMotionTransformer, SpatioTemporalMotionTransformer
from trainers import DDPMTrainer
from datasets import Text2MotionDataset
from tools.train import build_models

from mmcv.runner import get_dist_info, init_dist
from mmcv.parallel import MMDistributedDataParallel, MMDataParallel
import torch
import torch.distributed as dist
from mylib import animate3d, kit_bone, kit_kit_bone
from utils.motion_process import recover_from_ric

### Options:
1. If wanna test full model set opt.debug=False but need to comment out the UnifyLog<br>

In [3]:
class Temp:
    def __init__(self):
        print('mock:: opt')
opt = Temp()


# mock from /home/epinyoan/git/MotionDiffuse/text2motion/options/train_options.py
opt.num_layers = 8
opt.latent_dim = 512
opt.diffusion_steps = 1000
opt.no_clip = False
opt.no_eff = False
opt.num_epochs = 50
opt.lr = 2e-4
opt.batch_size = 32
opt.times = 1
opt.feat_bias = 1
opt.is_continue = False
opt.log_every = 50
opt.save_every_e = 5
opt.log_every = 5
opt.eval_every_e = 5
opt.save_latest = 500
opt.is_train = True

# mock from cmd
opt.name = 'TEMP'
opt.batch_size = 2 # [INFO] modified
opt.times = 50
opt.num_epochs = 1 # [INFO] modified
opt.dataset_name = 'kit' # [INFO] or t2m or kit
opt.num_layers = 8
opt.diffusion_steps = 1000
opt.data_parallel = True
opt.gpu_id = [0,1,2,3]
opt.device = torch.device("cuda")

# mock from /home/epinyoan/git/MotionDiffuse/text2motion/options/base_options.py
opt.checkpoints_dir = '../checkpoints'
opt.distributed = False

opt.save_root = pjoin(opt.checkpoints_dir, opt.dataset_name, opt.name)
opt.model_dir = pjoin(opt.save_root, 'model')
opt.meta_dir = pjoin(opt.save_root, 'meta')

if opt.dataset_name == 't2m':
    opt.data_root = '../data/HumanML3D' # change root to '../'
    opt.motion_dir = pjoin(opt.data_root, 'new_joint_vecs')
    opt.text_dir = pjoin(opt.data_root, 'texts')
    opt.joints_num = 22
    radius = 4
    fps = 20
    opt.max_motion_length = 196
    dim_pose = 263
    kinematic_chain = paramUtil.t2m_kinematic_chain
elif opt.dataset_name == 'kit':
    opt.data_root = '../data/KIT-ML' # change root to '../'
    opt.motion_dir = pjoin(opt.data_root, 'new_joint_vecs')
    opt.text_dir = pjoin(opt.data_root, 'texts')
    opt.joints_num = 21
    radius = 240 * 8
    fps = 12.5
    dim_pose = 251
    opt.max_motion_length = 196
    kinematic_chain = paramUtil.kit_kinematic_chain

opt.debug = True # [INFO] added
opt.corrupt = 'diffusion' # [INFO] added ['mask', 'diffusion']
dim_word = 300
mean = np.load(pjoin(opt.data_root, 'Mean.npy'))
std = np.load(pjoin(opt.data_root, 'Std.npy'))

train_split_file = pjoin(opt.data_root, 'train.txt')


train_dataset = Text2MotionDataset(opt, mean, std, train_split_file, opt.times)

mock:: opt


100%|██████████| 300/300 [00:00<00:00, 2400.62it/s]


### Debug Model

In [12]:
encoder = build_models(opt, dim_pose)
opt.is_train = False
trainer = DDPMTrainer(opt, encoder)
trainer.load('/home/epinyoan/git/MotionDiffuse/text2motion/checkpoints/kit/temp2/latest.tar')
mean = np.load('/home/epinyoan/git/MotionDiffuse/text2motion/checkpoints/kit/temp2/meta/mean.npy')
std = np.load('/home/epinyoan/git/MotionDiffuse/text2motion/checkpoints/kit/temp2/meta/std.npy')

In [15]:
from utils.plot_script import plot_3d_motion
from utils.motion_process import recover_from_ric
with torch.no_grad():
    caption = ["a person is jumping"]
    m_lens = torch.LongTensor([60]).cuda()
    pred_motions = trainer.generate(caption, m_lens, dim_pose)
    motion = pred_motions[0].cpu().numpy()
    motion = motion * std + mean
    title = caption[0] + " #%d" % motion.shape[0]
    joint = recover_from_ric(torch.from_numpy(motion).float(), opt.joints_num).numpy()
    if opt.dataset_name == 'kit':
        joint = joint/1000
print('joint:', joint[:, :, 0].mean(), joint[:, :, 1].mean(), joint[:, :, 2].mean())
plot_3d_motion(f'{opt.save_root}/{title}.gif', kinematic_chain, joint, title=title, fps=20)

  0%|          | 0/1000 [00:00<?, ?it/s]

joint: -0.06307936 0.5032406 0.06349116


In [22]:
batch_size = 1 #opt.batch_size
opt.max_motion_length = 50
x = torch.rand([batch_size, 50, dim_pose])
timesteps = torch.rand([batch_size])
length = torch.ones([batch_size], dtype=int)*4
text = ['individual flrx.']*batch_size
_ = encoder(x, timesteps, length=length, text=text)

### Test Inference

In [43]:
if opt.dataset_name == 't2m':
    trainer.load('../checkpoints/t2m/t2m_motiondiffuse/model/latest.tar')
    mean = np.load('../checkpoints/t2m/t2m_motiondiffuse/meta/mean.npy')
    std = np.load('../checkpoints/t2m/t2m_motiondiffuse/meta/std.npy')
elif opt.dataset_name == 'kit':
    trainer.load('../checkpoints/kit/kit_motiondiffuse/model/latest.tar')
    mean = np.load('../checkpoints/kit/kit_motiondiffuse/meta/mean.npy')
    std = np.load('../checkpoints/kit/kit_motiondiffuse/meta/std.npy')

In [39]:
opt.is_train = False
encoder = build_models(opt, dim_pose)
# encoder = MMDataParallel(
#             encoder.cuda(opt.gpu_id[0]), device_ids=opt.gpu_id)
trainer = DDPMTrainer(opt, encoder)
# trainer.load('../checkpoints/t2m/TEMP/model/latest.tar')
trainer.eval_mode()
trainer.to(opt.device)

In [72]:
with torch.no_grad():
    motion_length=40
    caption = ['A person walks in a circle clockwise and put hands in the air.']
    m_lens = torch.LongTensor([motion_length]).to(opt.device)
    pred_motions = trainer.generate(caption, m_lens, dim_pose, batch_size=1)
    motion = pred_motions[0].cpu().numpy()
#     motion = motion * std + mean

  0%|          | 0/1000 [00:00<?, ?it/s]

##### Raw data

In [3]:
mean1 = np.load('../data/KIT-ML/Mean.npy')
std1 = np.load('../data/KIT-ML/Std.npy')
id = 120
motion1 = train_dataset[id][1]
length = train_dataset[id][2]
motion1.shape, train_dataset[id][0], train_dataset[id][2]

((196, 251), 'A person walks forward at medium speed.', 67)

In [11]:
# _mean = mean.copy()
# motion_zero = np.zeros_like(motion)
# _mean[1:3] = 0 # xz velocity
# motion1 = motion_zero * std + _mean

In [49]:
# import glob
# for path in glob.glob('../data/KIT-ML/new_joint_vecs/*'):
#     motion1 = np.load(path)
#     break
# motion1.shape


(74, 251)

In [28]:
motion1 = motion1 * std1 + mean1
bones = kit_bone if opt.dataset_name == 'kit' else None
joint = recover_from_ric(torch.from_numpy(motion1).float(), opt.joints_num).numpy()
animate3d(joint[:length]/1000, BONE_LINK=bones, save_path=None) # 'init.html'

### Train

In [39]:
# del trainer
# del encoder
# torch.cuda.empty_cache()
_train_dataset = torch.utils.data.Subset(train_dataset, list(range(0, 8)))
encoder = build_models(opt, dim_pose)
# encoder = MMDataParallel(
#             encoder.cuda(opt.gpu_id[0]), device_ids=opt.gpu_id)
trainer = DDPMTrainer(opt, encoder)
trainer.train(_train_dataset)

  0%|          | 0/1 [00:00<?, ?it/s]

num batch: 1


100%|██████████| 1/1 [00:04<00:00,  4.30s/it]


In [4]:
trainer.train(_train_dataset)


  0%|                                                                                                                                                              | 0/1 [00:00<?, ?it/s]

num batch: 1


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.78s/it]


In [109]:
def get_mask(l, ratio, T=196):
    b = l.shape[0]
    mask = np.zeros((l.shape[0], T))
    for i, _l in enumerate(l):
        m_idx = np.arange(_l)
        np.random.shuffle(m_idx)
        m_idx = m_idx[:round(ratio*len(m_idx))]
        mask[i, m_idx] = 1
    return mask
    
get_mask(torch.tensor([2, 4, 3]), ratio=.5, T=5)

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 1., 0.],
       [1., 0., 1., 0., 0.]])

##### Visualize

In [36]:
mean2 = np.load('/home/epinyoan/git/MotionDiffuse/text2motion/checkpoints/kit/temp2/meta/mean.npy')
std2 = np.load('/home/epinyoan/git/MotionDiffuse/text2motion/checkpoints/kit/temp2/meta/std.npy')

In [40]:
trainer.load('/home/epinyoan/git/MotionDiffuse/text2motion/checkpoints/kit/temp2/latest.tar')
trainer.eval_mode()

ValueError: loaded state dict contains a parameter group that doesn't match the size of optimizer's group

In [ ]:
caption, motion, length = train_dataset[12]
motion = torch.from_numpy(motion).float().unsqueeze(0).to(trainer.device)
trainer.to(motion.device)
t = torch.ones(1, device=trainer.device) * 4
mask = trainer.get_mask(torch.tensor([length]), ratio=.5, T=length)
xf_proj, xf_out = trainer.encoder.encode_text(caption, trainer.device)
output_motion = trainer.encoder(motion, t, length=[length], text=[caption], xf_proj=xf_proj, xf_out=xf_out, mask=None)

In [90]:
motion2 = output_motion[0].detach().cpu().numpy() * std2 + mean2
bones = kit_bone if opt.dataset_name == 'kit' else None
joint = recover_from_ric(torch.from_numpy(motion2).float(), opt.joints_num).numpy()
animate3d(joint[:length]/1000, BONE_LINK=bones, save_path=None) # 'init.html'

In [95]:
motion1 = motion[0].detach().cpu().numpy() * std1 + mean1
motion2 = output_motion[0].detach().cpu().numpy() * std2 + mean2
bones = kit_bone if opt.dataset_name == 'kit' else None
joint1 = recover_from_ric(torch.from_numpy(motion1).float(), opt.joints_num).numpy()
joint2 = recover_from_ric(torch.from_numpy(motion2).float(), opt.joints_num).numpy()
print(joint1.shape, joint2.shape)

joint_original_forward = np.concatenate((joint1, joint2), axis=1)
animate3d(joint_original_forward[:length]/1000, BONE_LINK=kit_kit_bone, first_total_standard=60, save_path='Mask50Recon.html') # 'init.html'

(196, 21, 3) (196, 21, 3)
